<a href="https://colab.research.google.com/github/otanet/Quantum_Computing_Annealing_Machine_20211209/blob/main/number_partition_problem_20211209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 数の分割問題

$n$ 個の整数の集合 $A$ が

$$
A = \{a_0, a_1, \cdots, a_{n-1} \}
$$

で与えられているとします。$A$ に属する数を二つの集合 $A_0$ と $A_1$ に分割し、それぞれに属する数の和が等しくなる状況を探索します。

## 問題の定式化

まず、$A_0$および$A_1$に属する数の和をそれぞれ $S_0$, $S_1$とすると、
$$
S_0 = \sum_{a_i\in A_0}a_i, \quad S_1 = \sum_{a_i\in A_1}a_i
$$
となり、$S_1=S_0$ が求めるべき条件となります。

集合 $A$ に属する $n$ 個のそれぞれの数字に対応した $n$ 個のバイナリ変数$\{q_i\}$もしくはイジング変数$\{s_i\}$を用意します。つまり、
$$
q_i \in\{1, 0\}\quad (i=0, 1, \cdots, n-1) \quad \text{(Binary)}
$$
となるような$n$個の変数を用意するか、
$$
s_i \in\{1, -1\}\quad (i=0, 1, \cdots, n-1) \quad \text{(Ising)}
$$
となるような$n$個の変数を用意します。
これらの変数を用いて、$q_i = 1$ (あるいは $s_i=1$) の場合は $n_i$ は $A_1$ に属し、$q_i=0$ (あるいは $s_i=-1$) の場合は $n_i$ は $A_0$ に属するというように分割方法を表現することにします。  

### 目的関数
次に、目的関数を求めることを考えます。
目的関数とは、バイナリ変数、もしくはイジング変数の関数であり、求めるべき条件が満たされた場合に最小値をとるような関数です。
ここでは、 $S_0 = S_1$ の条件を満たす分割を探すため、目的関数を $(S_1 - S_0)^2$ とすると、条件が満たされた時に $0$ となり、最小値をとります。
したがって、バイナリ変数、またはイジング変数を使うと、目的関数 $f$ は以下のように書き下すことができます。

$$
\begin{align*}
f &= \left(S_1 - S_0\right)^2
= \left(\sum_{a_i\in A_1}a_i - \sum_{a_i\in A_0}a_i\right)^2\\
&= \left(\sum_{i=0}^{n-1}q_ia_i - \sum_{i=0}^{n-1}(1-q_i)a_i\right)^2 = \left(\sum_{i=0}^{n-1}(2q_i -1)a_i\right)^2 \quad \text{(Binary)}\\
&= \left(\sum_{i=0}^{n-1} a_i s_i \right)^2\quad \text{(Ising)}
\end{align*}
$$

1行目から2行目（3行目）への変換は、$q_i=1$ ($s_i=1$) または $q_i=0$ ($s_i=-1$) によって、$a_i$ は $A_1$ または $A_0$ に割り当てられることを使いました。$A_0$ と $A_1$ の分割でそれぞれの和が等しいと、$f=0$ となるので、$f$ の値を確認することで、条件を満たす分割がなされたかどうかを確かめることができます。

これで問題が定式化できたので、次は実際にプログラムを実行して問題を解きます。

## 二値多項式の生成

### 変数の生成

まず、数の集合 $A$ のそれぞれの数字に対応する$n$個の二値変数を用意します。

Amplify の `BinarySymbolGenerator()` を用いると、シンボリックな変数に対応したオブジェクトを生成する変数ジェネレータを作成することができます。
この変数ジェネレータを用いて変数配列を生成するには、`array`メソッドの引数に変数の数を渡します。
変数の数 $n$ は集合 $A$ の要素数とします。

In [ ]:
! pip install -q amplify

In [ ]:
from amplify import BinarySymbolGenerator

# 数の集合Aに対応する数のリスト
A = [2, 10, 3, 8, 5, 7, 9, 5, 3, 2]

# 変数の数
n = len(A)

# バイナリ変数を生成
gen = BinarySymbolGenerator()
q = gen.array(n)

In [ ]:
# 変数を確認
q

### 目的関数の構築

次に、数のリスト $A$ と先ほど生成した変数を用いて、目的関数 $\displaystyle\left(\sum_{i=0}^{n-1}(2q_i-1)a_i\right)^2$ を構築します。目的関数は二次以下の項からなる多項式になっていることが確認できます。

In [ ]:
# 目的関数の構築
f = 0
for i in range(n):
    f += (2 * q[i] - 1) * A[i]
f = f ** 2

# 目的関数を確認
f

Numpy に慣れている場合は、次のように配列同士の演算を用いると簡潔かつ高速に定式化が可能です。

In [ ]:
f = ((2 * q - 1) * A).sum() ** 2
f

## イジングマシンの実行

### クライアントの設定

イジングマシンのクライアントを作成し、必要なパラメータを設定します。
以下の設定では、最終的に取得できる解は一つになります。

In [ ]:
from amplify.client import FixstarsClient

# クライアントの設定
client = FixstarsClient()
client.token = "トークンを入力してください"
client.parameters.timeout = 1000  # タイムアウト1秒

次に、先ほど設定したクライアントからソルバーを作成し、`solve` 関数で問題を解きます。

In [ ]:
from amplify import Solver

solver = Solver(client)
result = solver.solve(f)

# 解が得られなかった場合、len(result.solutions) == 0
if len(result.solutions) == 0:
    raise RuntimeError("No solution was found")

energy = result.solutions[0].energy
values = result.solutions[0].values

In [ ]:
# エネルギー値 (f の最小値) を確認
print(f"f = {energy}")

# valuesを確認
# 変数 s_i (i=0, 1, ..., N-1) の値を格納した辞書
print(f"values = {values}")

$f$ の最小値が $0$ となる解となっているので、条件を満たす解が見つかったことがわかります。

見つかった解を元の変数 `q` と対応する形の配列にするには、 `decode` メソッドを使うと便利です。

In [ ]:
solution = q.decode(values)
solution

最後に、得られた解を元に、集合 $A$ の数字を二つのグループに分割します。

二つのリスト $A_0$ と $A_1$ を用意し、$0$ となっている $q_i$ に対応する数字は $A_0$に、そうでない数字は $A_1$ に割り振ります。

In [ ]:
A0 = sorted([A[idx] for idx, val in enumerate(solution) if val != 1])
A1 = sorted([A[idx] for idx, val in enumerate(solution) if val == 1])

print(f"A0 = {A0}")
print(f"A1 = {A1}")

$A_0$ と $A_1$ のそれぞれの数字の和が等しいことを確かめます。和は 27 となっていることが確認できます。

In [ ]:
print(f"A0 = {sum(A0)}, A1 = {sum(A1)}")

## 複数の解を得る方法

先ほどの問題では、解を一つだけ得る方法を紹介しました。しかしながら、この問題では、条件を満たす解は複数個見つけることができます。この分割問題の設定では、条件は目的関数が $0$ であることと等価であるため、条件を満たす解が複数個ある場合は、エネルギー値が $0.0$ である解が複数個あるということになります。一部のマシンは、同じエネルギーを持つ解を複数得ることが出来ます。Fixstars Annealing Engine の場合はパラメータ `client.parameters.outputs.duplicate` を `True` に設定することで複数の解が出力されます。

In [ ]:
from amplify.client import FixstarsClient
from amplify import Solver

client = FixstarsClient()
client.token = "トークンを入力してください"
client.parameters.timeout = 1000  # タイムアウト1秒
client.parameters.outputs.duplicate = True  # 同じエネルギー値の解を列挙するオプション（解が複数個あるため）

solver = Solver(client)
result = solver.solve(f)

解が複数個あることは以下のようにして確かめることができます。46個の解が見つかるはずです。

In [ ]:
len(result.solutions)

次に、見つけてきた複数の解を元の変数に代入して全ての分割を求めます。$(A_0, A_1)$ と $(A_1, A_0)$ の組合せを同一視する必要がある事に注意して下さい。

In [ ]:
partitions = set()

for s in result.solutions:
    solution = q.decode(s.values)

    A0 = tuple(sorted([A[idx] for idx, val in enumerate(solution) if val != 1]))
    A1 = tuple(sorted([A[idx] for idx, val in enumerate(solution) if val == 1]))

    # 同じ分割がすでにリストに含まれていない場合
    if (A1, A0) not in partitions:
        partitions.add((A0, A1))

for p in partitions:
    print(f"sum = {sum(p[0])}, {sum(p[1])}, partition: {p}")

## プログラミング課題

### 課題1

数の集合を変更して分割をおこなってください。もし等しい分割が可能ではない場合にはどのようになるか確認して下さい。

等しい分割が可能であるような数の集合の例：$\{1,2,3,5,8,13,21,34,55,89,144,233\}$<br>
等しい分割が可能でないような数の集合の例：$\{3,1,4,1,5,9,2,6,5,3,5,8,9,7,9\}$

### 課題2

上記ではバイナリ変数を用いた定式化を行いましたが、イジング変数を用いた定式化を行い問題を解いて下さい。

### 課題3

[部分和問題](https://ja.wikipedia.org/wiki/%E9%83%A8%E5%88%86%E5%92%8C%E5%95%8F%E9%A1%8C) とは、$n$ 個の整数の集合 $A=\{a_0, a_1, \cdots, a_n\}$ と整数 $N$ が与えられた際に、$A$ の部分集合の和を $N$ に等しくなるように選ぶことができるかを判定する問題です。数の分割問題の定式化を参考にして部分和問題を解いて下さい。

問題の例：$A=\{1000000,200000,30000,4000,500,60,7\}, \quad N=1004507$